In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from collections import defaultdict

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TextVectorization, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
df = pd.read_csv("/content/book_data.csv")
df

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
...,...,...,...,...,...,...,...,...,...,...,...,...
54296,Howard Megdal,"In this fearless and half-crazy story, Howard ...",NaN,Hardcover,9.78161E+12,256 pages,3.37,27,9,Taking the Field: A Fan's Quest to Run the Tea...,Sports|Baseball|Sports and Games|Sports|Nonfic...,https://images.gr-assets.com/books/1312074392l...
54297,Howard Megdal,From the icons of the game to the players who ...,NaN,Hardcover,9.78006E+12,256 pages,3.97,34,5,"The Baseball Talmud: Koufax, Greenberg, and th...",Nonfiction|Sports and Games|Sports,https://images.gr-assets.com/books/1348841629l...
54298,Howard Megdal,NaN,NaN,Kindle Edition,NaN,NaN,3.66,32,3,"Wilpon's Folly - The Story of a Man, His Fortu...",Sports|Baseball|Abandoned,https://images.gr-assets.com/books/1394277097l...
54299,Mimi Baird|Eve Claxton,"Soon to be a major motion picture, from Brad P...",NaN,Hardcover,9.7808E+12,272 pages,3.82,867,187,He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,https://images.gr-assets.com/books/1403192135l...


In [ ]:
df.dropna(subset=["genres", "book_desc"], inplace=True)

In [ ]:
data = df.copy()

Calculating the frequency of each genre

In [ ]:
def genre_count(x):
  '''returns the number of genres associated with the book
  '''
  try:
      return len(x.split('|'))
  except:
      return 0

data['genre_count'] = data['genres'].map(lambda x: genre_count(x))

make a genre columns into a list of all genres

In [ ]:
def genre_listing(x):
    try:
        lst = [genre for genre in x.split("|")]
        return lst
    except:
        return []

data['genre_list'] = data['genres'].map(lambda x: genre_listing(x))

In [ ]:
genre_dict = defaultdict(int)
for index, row in data.iterrows():
  book_genres = row["genre_list"]
  if type(book_genres) == list:
    for genre in book_genres:
      genre_dict[genre] += 1

In [ ]:
len(genre_dict)

864

In [ ]:
genre_pd = pd.DataFrame.from_records(sorted(genre_dict.items(), key=lambda x:x[1], reverse=True),
                                     columns=['genre', 'count'])
genre_pd[:20]

,genre,count
0,Fiction,25736
1,Fantasy,23583
2,Romance,18636
3,Young Adult,11251
4,Historical,10789
5,Paranormal,7994
6,Mystery,7902
7,Nonfiction,7598
8,Science Fiction,6780
9,Historical Fiction,6399


In [ ]:
def determine_fiction(x):
    lower_list = [genre.lower() for genre in x]
    if 'fiction' in lower_list:
        return 'fiction'
    elif 'nonfiction' in lower_list:
        return 'nonfiction'
    else:
        return 'others'
data['label'] = data['genre_list'].apply(determine_fiction)

In [ ]:
data.label.value_counts()

fiction       25736
others        16843
nonfiction     7598
Name: label, dtype: int64

In [ ]:
data = data[(data.label == "fiction") | (data.label == "nonfiction")]

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
data['book_desc'] = data['book_desc'].apply(clean_text)

In [ ]:
max_length = 200  # Define the maximum length of sequences

vectorizer = TextVectorization(
    max_tokens=10000,
    output_sequence_length=max_length,
    output_mode="int"
    )

text_data = df['book_desc'].values  # Assuming 'summary' column contains the book summaries
vectorizer.adapt(text_data)

In [ ]:
def stratified_split(df, target, val_percent=0.2):
    classes=list(df[target].unique())
    train_idxs, val_idxs = [], []
    for c in classes:
        idx=list(df[df[target]==c].index)
        np.random.shuffle(idx)
        val_size=int(len(idx)*val_percent)
        val_idxs+=idx[:val_size]
        train_idxs+=idx[val_size:]
    return train_idxs, val_idxs

In [ ]:
data

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url,genre_count,genre_list,label
0,Suzanne Collins,winning will make you famous losing means cert...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...,6,"[Young Adult, Fiction, Science Fiction, Dystop...",fiction
1,J.K. Rowling|Mary GrandPré,there is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...,3,"[Fantasy, Young Adult, Fiction]",fiction
2,Harper Lee,the unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...,6,"[Classics, Fiction, Historical, Historical Fic...",fiction
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,è cosa ormai risaputa che a uno scapolo in pos...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...,3,"[Classics, Fiction, Romance]",fiction
4,Stephenie Meyer,about three things i was absolutely positive f...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...,8,"[Young Adult, Fantasy, Romance, Paranormal, Va...",fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54294,Siri Hustvedt,a brilliant provocative novel about an artist ...,First Simon & Schuster hardcover edition,Hardcover,9.78148E+12,368 pages,3.67,5827,816,The Blazing World,Fiction|Art|Feminism|Contemporary|Literary Fic...,https://images.gr-assets.com/books/1418103958l...,6,"[Fiction, Art, Feminism, Contemporary, Literar...",fiction
54295,Avi Steinberg,avi steinberg is stumped after defecting from ...,NaN,Hardcover,9.78039E+12,399 pages,3.51,3717,661,Running the Books: The Adventures of an Accide...,Nonfiction|Autobiography|Memoir|Biography|Writ...,https://images.gr-assets.com/books/1320533033l...,7,"[Nonfiction, Autobiography, Memoir, Biography,...",nonfiction
54296,Howard Megdal,in this fearless and half crazy story howard m...,NaN,Hardcover,9.78161E+12,256 pages,3.37,27,9,Taking the Field: A Fan's Quest to Run the Tea...,Sports|Baseball|Sports and Games|Sports|Nonfic...,https://images.gr-assets.com/books/1312074392l...,5,"[Sports, Baseball, Sports and Games, Sports, N...",nonfiction
54297,Howard Megdal,from the icons of the game to the players who ...,NaN,Hardcover,9.78006E+12,256 pages,3.97,34,5,"The Baseball Talmud: Koufax, Greenberg, and th...",Nonfiction|Sports and Games|Sports,https://images.gr-assets.com/books/1348841629l...,3,"[Nonfiction, Sports and Games, Sports]",nonfiction


In [ ]:
data["label"] = data["label"].map({"fiction":0, "nonfiction":1})

In [ ]:
train_idx, val_idx = stratified_split(data, "label")

In [ ]:
train_data = data.loc[train_idx]
val_data = data.loc[val_idx]

y_train = train_data.pop("label")
y_valid = val_data.pop("label")

In [ ]:
train_data["book_desc"].values

array(['4 girls 3 summers and 1 pair of pants for the first time ever all three novels in the 1 new york times bestselling series are available in this terrific boxed set which includes an exclusive magnet frame and magnets of the pants',
       'neal asher whom tor introduced to the american audience with gridlinked takes us deeper into his unique universe with an even more remarkable second novel the skinner on the planet spatterjay arrive three travelers janer acting as the eyes of the hornet hive mind on a mission not yet revealed to him erlin searching for ambel the ancient sea captain who can teach her how to live and sable keech on a vendetta he cannot abandon though he himself has been dead for 700 years this remote world is mostly ocean and it is a rare visitor who ventures beyond the safety of the island dome outside it only the native hoopers dare risk the voracious appetites of the planet s wildlife but somewhere out there is spatterjay hoop and keech will not rest until he

In [ ]:
train_data = vectorizer(train_data["book_desc"].values)
val_data = vectorizer(val_data["book_desc"].values)

In [ ]:
sequences = vectorizer(text_data)
vocab_size = len(vectorizer.get_vocabulary())

In [ ]:
def book_LSTM():
    model = Sequential()
    embedding_dim = 256
    model.add(Embedding(vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(Bidirectional(
        LSTM(256, return_sequences=True)
        ))
    model.add(Dropout(0.2))
    model.add(Bidirectional(
        LSTM(256)
        ))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

    return model

In [ ]:
book_model = book_LSTM()

book_model.fit(train_data,
    y_train,
    validation_data=(val_data, y_valid),
    batch_size=64,
    epochs=5)

Epoch 1/5
417/417 [==============================] - 1105s 3s/step - loss: 0.4493 - accuracy: 0.8137 - val_loss: 0.2980 - val_accuracy: 0.8897
Epoch 2/5
417/417 [==============================] - 1104s 3s/step - loss: 0.2189 - accuracy: 0.9192 - val_loss: 0.2329 - val_accuracy: 0.9116
Epoch 3/5
417/417 [==============================] - 1099s 3s/step - loss: 0.1537 - accuracy: 0.9457 - val_loss: 0.3212 - val_accuracy: 0.9079
Epoch 4/5
417/417 [==============================] - 1104s 3s/step - loss: 0.1662 - accuracy: 0.9418 - val_loss: 0.2755 - val_accuracy: 0.9154
Epoch 5/5
333/417 [======================>.......] - ETA: 3:24 - loss: 0.0932 - accuracy: 0.9682

In [ ]:
book_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 256)          2560000   
                                                                 
 bidirectional (Bidirectiona  (None, 200, 512)         1050624   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200, 512)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 5,186,049
Trainable params: 5,186,049
Non-

In [ ]:
test_data = pd.read_csv("/content/book_data18.csv")
test_data

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Kristin Hannah,"Alaska, 1974.Unpredictable. Unforgiving. Untam...",NaN,Kindle Edition,NaN,435 pages,4.33,146505,17438,The Great Alone,Fiction|Historical|Historical Fiction|Audioboo...,https://images.gr-assets.com/books/1501852423l...
1,Madeline Miller,"In the house of Helios, god of the sun and mig...",NaN,Hardcover,9.780000e+12,393 pages,4.34,61357,9369,Circe,Fantasy|Fiction|Fantasy|Mythology|Historical|H...,https://images.gr-assets.com/books/1508879575l...
2,Holly Black,Of course I want to be like them. They’re beau...,NaN,Hardcover,9.780000e+12,370 pages,4.18,62897,11948,The Cruel Prince,Fantasy|Young Adult|Fairies|Fae,https://images.gr-assets.com/books/1493047153l...
3,A.J. Finn,Anna Fox lives alone—a recluse in her New York...,NaN,Kindle Edition,NaN,449 pages,3.97,175678,20645,The Woman in the Window,Mystery|Thriller|Fiction|Suspense|Thriller|Mys...,https://images.gr-assets.com/books/1528225499l...
4,Tara Westover,An unforgettable memoir in the tradition of Th...,NaN,Hardcover,9.780000e+12,334 pages,4.47,128337,16424,Educated,Nonfiction|Autobiography|Memoir|Biography,https://images.gr-assets.com/books/1506026635l...
...,...,...,...,...,...,...,...,...,...,...,...,...
1136,Alyssa Brandon,A teen werewolf finally meets her destined sou...,NaN,Hardcover,9.780000e+12,304 pages,3.33,359,88,Bound to You,Fantasy|Paranormal|Romance|Fantasy|Shapeshifte...,https://images.gr-assets.com/books/1508489753l...
1137,Crystal Hana Kim,"An emotionally riveting debut novel about war,...",NaN,Hardcover,9.780000e+12,417 pages,3.69,1255,259,If You Leave Me,Historical|Historical Fiction|Fiction|Historic...,https://images.gr-assets.com/books/1508962755l...
1138,Leslie Cohen,This is the classic tale of boy meets girl: Gi...,NaN,Kindle Edition,NaN,320 pages,3.40,1300,272,This Love Story Will Self-Destruct,Romance|Fiction|Womens Fiction|Chick Lit|Conte...,https://images.gr-assets.com/books/1503757123l...
1139,Sarah Beth Durst,The battle between vicious spirits and strong-...,NaN,Hardcover,9.780000e+12,419 pages,3.86,907,205,The Queen of Sorrow,Fantasy|Young Adult|Adult,https://images.gr-assets.com/books/1513185111l...


In [ ]:
test_data.dropna(subset=["book_desc", "genres"], inplace=True)

In [ ]:
test_data["book_desc"] = test_data["book_desc"].apply(clean_text)
test_data['genre_list'] = test_data['genres'].map(lambda x: genre_listing(x))
test_data['label'] = test_data['genre_list'].apply(determine_fiction)
test_data = test_data[(test_data.label == "fiction") | (test_data.label == "nonfiction")]
test_data["label"] = test_data["label"].map({"fiction":0, "nonfiction":1})

y_test = test_data.pop("label")
test_data = vectorizer(test_data["book_desc"].values)

<ipython-input-56-de24bd579bf2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["label"] = test_data["label"].map({"fiction":0, "nonfiction":1})


In [ ]:
def test_vectorizer(text):
    a = clean_text(text)
    a = vectorizer(a)
    a = np.reshape(a, (1, max_length))
    return a

In [ ]:
results = book_model.evaluate(test_data, y_test, batch_size=64)

11/11 [==============================] - 3s 255ms/step - loss: 0.1222 - accuracy: 0.9697


In [ ]:
results

[0.14253166317939758, 0.9531013369560242]

In [ ]:
def reviewBook(model,text):
    labels = ['fiction', 'nonfiction']
    a = clean_text(text)
    a = vectorizer(a)
    a = np.reshape(a, (1, max_length))
    output = model.predict(a, batch_size=1)
    score = (output>0.5)*1
    pred = score.item()
    return labels[pred]

In [ ]:
text = "In a post-apocalyptic future humans live in peace with androids in a domed city. However, a strange series of murders has intruded into bored inspector Re-l Mayer's life."

In [ ]:
reviewBook(book_model, text)

1/1 [==============================] - 1s 1s/step


'fiction'

In [ ]:
text_2 = "from the icons of the game to the players who got their big break but never quite broke through the baseball talmud provides a wonderful historical narration of major league jewish baseball in america all the stats the facts the stories and the often unheralded glory the baseball talmud reveals that there is far more to jewish baseball than hank greenberg s powerful slugging and sandy koufax s masterful control from ausmus to zinn berg to kinsler holtzman to yeager and many others megdal draws upon the lore and the little known details that increase our enjoyment of the game including which jewish player spent a portion of his retirement as a spywho received 50 000"

In [ ]:
reviewBook(book_model, text_2)

1/1 [==============================] - 0s 64ms/step


'nonfiction'

In [ ]:
book_model.save("book_model.h5")